## Deliverable 2. Create a Customer Travel Destinations Map.

In [17]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from g_key import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Omboue,-1.5746,9.2618,74.75,89,95,6.24,GA,2022-08-21 19:08:08
1,1,Lamu,-2.2717,40.9020,75.42,81,0,16.96,KE,2022-08-21 19:08:08
2,2,Broken Hill,-31.9500,141.4333,51.49,59,10,10.74,AU,2022-08-21 19:08:09
3,3,Yar-Sale,66.8333,70.8333,54.57,94,100,5.32,RU,2022-08-21 19:08:09
4,4,Albany,42.6001,-73.9662,87.03,47,100,3.00,US,2022-08-21 19:07:31


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip?'))
max_temp = float(input('What is the maximum temperature you would like for your trip?'))

What is the minimum temperature you would like for your trip?65
What is the maximum temperature you would like for your trip?85


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Omboue,-1.5746,9.2618,74.75,89,95,6.24,GA,2022-08-21 19:08:08
1,1,Lamu,-2.2717,40.9020,75.42,81,0,16.96,KE,2022-08-21 19:08:08
8,8,Namatanai,-3.6667,152.4333,77.65,83,81,5.17,PG,2022-08-21 19:08:11
9,9,Nikolskoye,59.7035,30.7861,70.81,68,34,6.02,RU,2022-08-21 19:08:11
11,11,Bengkulu,-3.8004,102.2655,74.14,87,100,3.76,ID,2022-08-21 19:08:12
17,17,Marsa Matruh,31.3525,27.2453,79.97,71,0,14.09,EG,2022-08-21 19:08:14
23,23,Hithadhoo,-0.6000,73.0833,82.22,77,98,11.59,MV,2022-08-21 19:08:16
25,25,Sao Filipe,14.8961,-24.4956,77.14,88,100,7.74,CV,2022-08-21 19:08:17
26,26,Lata,40.1629,-8.3327,70.25,77,0,6.67,PT,2022-08-21 19:08:17
30,30,Pringsewu,-5.3582,104.9744,71.98,97,100,2.08,ID,2022-08-21 19:08:19


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       1
Date          0
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna(axis="index", how="any")
clean_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Omboue,-1.5746,9.2618,74.75,89,95,6.24,GA,2022-08-21 19:08:08
1,1,Lamu,-2.2717,40.9020,75.42,81,0,16.96,KE,2022-08-21 19:08:08
8,8,Namatanai,-3.6667,152.4333,77.65,83,81,5.17,PG,2022-08-21 19:08:11
9,9,Nikolskoye,59.7035,30.7861,70.81,68,34,6.02,RU,2022-08-21 19:08:11
11,11,Bengkulu,-3.8004,102.2655,74.14,87,100,3.76,ID,2022-08-21 19:08:12


In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Wind Speed", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Wind Speed,Lat,Lng,Hotel Name
0,Omboue,GA,74.75,6.24,-1.5746,9.2618,
1,Lamu,KE,75.42,16.96,-2.2717,40.9020,
8,Namatanai,PG,77.65,5.17,-3.6667,152.4333,
9,Nikolskoye,RU,70.81,6.02,59.7035,30.7861,
11,Bengkulu,ID,74.14,3.76,-3.8004,102.2655,
17,Marsa Matruh,EG,79.97,14.09,31.3525,27.2453,
23,Hithadhoo,MV,82.22,11.59,-0.6000,73.0833,
25,Sao Filipe,CV,77.14,7.74,14.8961,-24.4956,
26,Lata,PT,70.25,6.67,40.1629,-8.3327,
30,Pringsewu,ID,71.98,2.08,-5.3582,104.9744,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        hotel_df.loc[index,"Hotel Name"]=np.nan
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [19]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df

,City,Country,Max Temp,Wind Speed,Lat,Lng,Hotel Name
0,Omboue,GA,74.75,6.24,-1.5746,9.2618,Hotel Olako
1,Lamu,KE,75.42,16.96,-2.2717,40.9020,Lamu House
8,Namatanai,PG,77.65,5.17,-3.6667,152.4333,Seagulls Inn Namatani
9,Nikolskoye,RU,70.81,6.02,59.7035,30.7861,Tourist House - Sablino
11,Bengkulu,ID,74.14,3.76,-3.8004,102.2655,Grage Hotel Bengkulu
...,...,...,...,...,...,...,...
528,Gizo,SB,80.76,8.59,-8.1030,156.8419,Imagination Island
529,Pekan,MY,75.31,5.91,3.4836,103.3996,Pekan Budget Hotel
530,Rolla,US,83.21,9.22,37.9514,-91.7713,Holiday Inn Express & Suites Rolla - Univ of M...
531,Eydhafushi,MV,81.90,15.52,5.1033,73.0708,Soneva Fushi


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Wind Speed</dt><dd>{Wind Speed}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))